# 파이널 프로젝트 : 은행 이탈 고객 예측



- **분류**
- 회귀



- 데이터
    - URL : https://www.kaggle.com/competitions/playground-series-s4e1
    - train : 훈련 데이터 세트
    - test : 테스트 데이터 세트
    - sample_submission : 올바른 형식의 샘플 제출 파일

- 평가 지표
    - ROC Curve


- 변수
    - Features(X)
        - ID : 순번
        - Customer ID: 각 고객의 고유 식별번호
        - Surname: 고객의 성
        - Credit Score: 고객의 신용점수
        - Geography: 고객이 거주하는 국가
        - Gender: 고객의 성별
        - Age: 고객의 나이
        - Tenure: 고객이 은행을 이용한 기간
        - Balance: 고객의 계좌 잔액
        - NumOfProducts: 고객이 이용하는 은행 상품의 수(ex. 예금,적금)
        - HasCrCard: 신용카드 보유 여부
        - IsActiveMember: 활성 회원 여부
        - EstimatedSalary: 고객의 예상 연봉
    - Target(Y)
        - Exited: 고객 이탈 여부







## 1.전처리

### 먼저 설치 후 재시작해야하는 패키지

In [ ]:
!pip install pycaret
!pip install optuna
!pip install catboost

### *패키지

In [ ]:
# 기본 패키지
import pandas as pd
import numpy as np

# 설정
pd.set_option('display.max_columns', None)  # 최대 컬럼 설정

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 출력 관련(이미지 표시)
from IPython.display import Image

# 계층적 클러스터링
import scipy.cluster.hierarchy as sch  # 계층적 군집화 알고리즘

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder  # 레이블 인코딩
# pd.get_dummies로 OneHotEncoder 대체

# 데이터 분할
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

# 다중공선성 확인(VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# PyCaret: AutoML (자동화된 머신러닝 워크플로우, 모델 선택)
from pycaret.classification import *

# Optuna: 하이퍼파라미터 최적화
import optuna
from optuna.samplers import TPESampler  # 샘플링 알고리즘 (TPE)

# 머신러닝 모델
## GBR
from sklearn.ensemble import GradientBoostingClassifier
## AdaBoost
from sklearn.ensemble import AdaBoostClassifier
## LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb
## DecisionTree
from sklearn.tree import DecisionTreeClassifier
## XGBoost
from xgboost import XGBClassifier
import xgboost as xgb
## CatBoost
from catboost import CatBoostClassifier, Pool # pool : catboost에서 데이터 관리위한 특수 클래스(데이터 컨테이너), 범주 변수 자동 인식, 데이터 빠른 처리 등
from catboost.utils import eval_metric

# 성능 평가 지표
from sklearn.metrics import roc_auc_score  # ROC AUC 점수 계산

# 모델 앙상블
from sklearn.ensemble import VotingClassifier  # 앙상블 학습을 위한 VotingClassifier

# 조합 생성 함수
from itertools import combinations

# 변수 중요도
from sklearn.inspection import permutation_importance

### 피쳐요약표

- 타입
- 고유값수
- 타입
- 고유값수
- 결측치수
- 중복치수
- 최소값
- 최대값
- 예시1
- 예시2

In [ ]:
# 피쳐요약표
def feature_summary(df):
  df_temp = pd.DataFrame()
  df_temp['타입'] = df.dtypes
  df_temp['고유값수'] = df.nunique()
  df_temp['결측치수'] = df.isnull().sum()
  df_temp['중복치수'] = df.apply(lambda x: x.duplicated().sum())
  df_temp['최소값'] = df.min()
  df_temp['최대값'] = df.max()
  df_temp['예시1'] = df.iloc[0]
  df_temp['예시2'] = df.iloc[1]
  return df_temp

### 데이터 불러오기
- train
- test
- submission

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/train.csv')
test = pd.read_csv('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/sample_submission.csv')

#### 데이터 파악

##### 데이터 크기

- train
    - 데이터 : 165034
    - 변수 : 14
- test
    - 데이터 : 110023
    - 변수 : 13
- submission
    - 데이터 : 110023
    - 변수 : 2

In [ ]:
for i in [train, test,submission] :
  print(i.shape)

##### 데이터 타입,결측
  - 결측치
      - 0개
  - 타입
      - Int64, float64 : 11개
      - object : 3개
          - Surname : 고객의 성
          - Geography : 고객이 거주하는 국가
          - Gender : 고객의 성별
- **test**
  - 결측치
      - 0개
  - 타입
      - train과 동일(Exited 제외)
- **submission**
  - 결측치
      - 0개
  - 타입
      - int64, float64

In [ ]:
# 피처요약표
feature_summary(train)
feature_summary(test)
feature_summary(submission)

#### 시각화
- **Barplot**
    - Surname : 범주형 변수이지만 고유값 수가 2797개로 부적합
    - Geography
    - Gender
    - NumOfProducts
    - HasCrCard
    - IsActiveMember
    - Exited
- **Histplot, 밀도추정그래프, BoxPlot, ScatterPlot**
    - CreditScore
    - Age
    - Tenure
    - Balance
    - EstimatedSalary

- **상관관계 히트맵**

##### 시각화 함수 생성

In [ ]:
# 1. BarPlot
def custom_barplot(ax, count):
    for bar in ax.patches:                                                            # ax.patches : 그래프 그려지는 모든 도형 요소
        if bar.get_height() == 0: continue                                            # 막대의 높이가 0이면 건너뜀
        rate = bar.get_height() / count * 100                                         # 백분율 계산

        ax.text(x=bar.get_x() + bar.get_width() / 2,                                  # 막대의 중앙 위치 : bar.get_x(막대의 왼쪽 x좌표), bar.get_width(막대의 너비 = 가로 길이)
                y=bar.get_height() + count * 0.005,                                   # 막대 위쪽에 텍스트 위치 : bar.get_height(막대의 높이 = 세로 길이)
                s=f'{rate:1.1f}%', ha='center')                                       # 비율 텍스트 추가

    ax.set_ylabel('')                                                                 # y축 레이블 제거
    return ax                                                                         # 수정된 Axes 객체 반환

# 2. Histogram
def custom_histogram(ax, data):
    mean = data.mean()                                                                # 데이터 평균 계산
    ax.text(x=ax.get_xlim()[0] + 0.05 * (ax.get_xlim()[1] - ax.get_xlim()[0]),        # 왼쪽 여백
            y=max([bar.get_height() for bar in ax.patches]) * 0.85,                   # y 위치 조정
            s=f'Mean: {mean:.2f}', ha='left', color='red')                            # 평균값 표시

    ax.set_ylabel('')                                                                 # y축 레이블 제거
    return ax                                                                         # 수정된 Axes 객체 반환

# 3. Box Plot
def custom_boxplot(ax, data):
    y_median = np.median(data)                                                        # 중앙값 계산
    ax.text(x=1.02, y=y_median, s=f'Median: {y_median:.2f}', ha='left', color='red')  # 중앙값 표시


##### train 시각화


In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(15, 20))

# Credit Score
sns.histplot(train['CreditScore'], bins=30, kde=True, ax=axes[0, 0])
custom_histogram(axes[0, 0], train['CreditScore'])
axes[0, 0].set_title('Credit Score Distribution')

# Geography
ax_geo = sns.countplot(data=train, x='Geography', hue='Exited', ax=axes[0, 1])
custom_barplot(ax_geo, train.shape[0])
axes[0, 1].set_title('Geography Distribution')

# Gender
ax_gender = sns.countplot(data=train, x='Gender', hue='Exited', ax=axes[1, 0])
custom_barplot(ax_gender, train.shape[0])
axes[1, 0].set_title('Gender Distribution')

# Age
sns.histplot(train['Age'], bins=30, kde=True, ax=axes[1, 1])
custom_histogram(axes[1, 1], train['Age'])
axes[1, 1].set_title('Age Distribution')

# Tenure
ax_tenure = sns.countplot(data=train, x='Tenure', hue='Exited', ax=axes[2, 0])
custom_barplot(ax_tenure, train.shape[0])
axes[2, 0].set_title('Tenure Distribution')

# Balance
sns.histplot(train['Balance'], bins=30, kde=True, ax=axes[2, 1])
custom_histogram(axes[2, 1], train['Balance'])
axes[2, 1].set_title('Balance Distribution')

# NumOfProducts
ax_num_products = sns.countplot(data=train, x='NumOfProducts', hue='Exited', ax=axes[3, 0])
custom_barplot(ax_num_products, train.shape[0])
axes[3, 0].set_title('NumOfProducts Distribution')

# HasCrCard
ax_has_crcard = sns.countplot(data=train, x='HasCrCard', hue='Exited', ax=axes[3, 1])
custom_barplot(ax_has_crcard, train.shape[0])
axes[3, 1].set_title('HasCrCard Distribution')

# IsActiveMember
ax_active_member = sns.countplot(data=train, x='IsActiveMember', hue='Exited', ax=axes[4, 0])
custom_barplot(ax_active_member, train.shape[0])
axes[4, 0].set_title('IsActiveMember Distribution')

# Estimated Salary
sns.histplot(train['EstimatedSalary'], bins=30, kde=True, ax=axes[4, 1])
custom_histogram(axes[4, 1], train['EstimatedSalary'])
axes[4, 1].set_title('Estimated Salary Distribution')

# Exited
ax_exited = sns.countplot(data=train, x='Exited', ax=axes[5, 0])
custom_barplot(ax_exited, train.shape[0])
axes[5, 0].set_title('Exited Distribution')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Credit Score
sns.boxplot(y=train['CreditScore'], ax=axes[0, 0])
custom_boxplot(axes[0, 0], train['CreditScore'])
axes[0, 0].set_title('Credit Score')

# Age
sns.boxplot(y=train['Age'], ax=axes[0, 1])
custom_boxplot(axes[0, 1], train['Age'])
axes[0, 1].set_title('Age')

# Balance
sns.boxplot(y=train['Balance'], ax=axes[1, 0])
custom_boxplot(axes[1, 0], train['Balance'])
axes[1, 0].set_title('Balance')

# Estimated Salary
sns.boxplot(y=train['EstimatedSalary'], ax=axes[1, 1])
custom_boxplot(axes[1, 1], train['EstimatedSalary'])
axes[1, 1].set_title('Estimated Salary')

plt.tight_layout()
plt.show()

###### train 시각화 해석


- **BarPlot, HistPlot 해석**
    - **Credit Score(신용점수)**
        - 평균
            - 656.45
        - 분포
            - 대체로 정규분포 형태
            - 신용 점수 분포가 넓게 퍼져있어 다양한 점수 분포
            - 신용점수가 600-700 사이에 가장 많은 고객이 분포

    - **Geography(고객 거주 국가)**
        - 분포
            - France가 전체의 57.1%로 가장 많고, 그 다음 Spain 22%, Germany 20.9% 순서
        - 이탈
            - 세 국가 모두 이탈하지 않은 고객(0)이 이탈 고객(1)보다 많다
                - 프랑스는 이탈 비율이 낮고, 독일은 상대적으로 높은 이탈율을 보인다

    - **Gender(고객 성별)**
        - 분포
            - 남성이 약 56.5%, 여성이 43.6%로 대략 6:4 비율
        - 이탈
            - 남성, 여성 모두 이탈하지 않은 사람(0)이 이탈자(1)보다 많다
                - 남성보다 여성이 이탈 비율이 더 높다

    - **Age(고객 연령)**
        - 평균
            - 38.13
        - 분포
            - 대체로 정규분포 형태
            - 30,40대에 주 고객층이 모여있다
            - 나이가 많을 수록 고객 수가 급격히 감소하는 경향
                - 젊은 고객층(30-40대)에서 고객수가 집중

    - **Tenure(은행 이용 기간)**
        - 분포
            - 모든 기간에 고객수가 고르게 분포되어 있지만, 0년과 10년에 소폭 감소
        - 이탈
            - 이용 기간에 관계없이 이탈하지 않은 사람이(0)이 이탈자(1) 보다 많다

    - **Balance(계좌잔액)**
        - 평균
            - 55478.09
        - 분포
            - 0인 경우가 가장 많으며, 0을 제외한 나머지 값들은 정규분포에 가깝게 분포
            - 주로 50,000~200,000 구간에 집중

    - **NumOfProducts(고객 이용 은행 상품 수)**
        - 분포
            - 1개 또는 2개 상품을 보유한 고객이 대부분
                - 1개(46.9%), 2개(51.1%)
        - 이탈
            - 상품 개수가 1개인 고객의 이탈 비율이 상대적으로 높다

    - **HasCrCard(신용카드 보유 여부)**
        - 분포
            - 신용카드 보유 고객(1)이 75.4%, 비보유 고객(0)이 24.6%의 분포로 보유한 고객이 더 많다
        - 이탈
            - 신용카드 보유 여부와 관계없이 이탈하지 않은 고객이 더 많다.

    - **IsActiveMemebr(활성 회원 여부)**
        - 분포
          - 비활성회원이 50.2%, 활성회원이 49.7%로 차이가 매우 적었다.
        - 이탈
            - 비활성 회원의 이탈율이 더 높다.

    - **EstimatedSalary(추정 연봉)**
        - 평균
            - 112574.82
        - 분포
            - 급여가 0에서 200000까지 고르게 분포
                - 주요 구간 : 50000 - 175000

    - **Exited(이탈 여부)**
        - 분포
            - 전체 고객 중 이탈하지 않은 사람(0)이 78.8%, 이탈자(1)는 21.2%로 대략 8:2 비율

- BoxPlot
    - CreditScore
        - 이상치
            - O
            - 신용 점수 하단에 이상치 다수 존재
                - 특히 400이하 낮은 점수 고객, 점수가 낮은 일부 고객층 포함
        - 중앙값
            - 659
        - 분포
            - 약간 오른쪽으로 치우친 분포
                - 왼쪽꼬리

    - Age
        - 이상치
            - O
            - 60 이상의 나이에서 이상치 다수 존재
                - 특히 80세 이상 고객이 드물게 포함, 고령층 소수 포함
        - 중앙값
            - 37
        - 분포
            - 오른쪽 긴 꼬리
    - Balance
        - 이상치
            - X
        - 중앙값
            - 0
    - EstimatedSalary
        - 이상치
            - X
        - 중앙값
            - 117,948

##### test 시각화


In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 20))

# Credit Score
sns.histplot(test['CreditScore'], bins=30, kde=True, ax=axes[0, 0])
custom_histogram(axes[0, 0], test['CreditScore'])
axes[0, 0].set_title('Credit Score Distribution')

# Geography
ax_geo = sns.countplot(data=test, x='Geography', ax=axes[0, 1])
custom_barplot(ax_geo, test.shape[0])
axes[0, 1].set_title('Geography Distribution')

# Gender
ax_gender = sns.countplot(data=test, x='Gender', ax=axes[1, 0])
custom_barplot(ax_gender, test.shape[0])
axes[1, 0].set_title('Gender Distribution')

# Age
sns.histplot(test['Age'], bins=30, kde=True, ax=axes[1, 1])
custom_histogram(axes[1, 1], test['Age'])
axes[1, 1].set_title('Age Distribution')

# Tenure
ax_tenure = sns.countplot(data=test, x='Tenure', ax=axes[2, 0])
custom_barplot(ax_tenure, test.shape[0])
axes[2, 0].set_title('Tenure Distribution')

# Balance
sns.histplot(test['Balance'], bins=30, kde=True, ax=axes[2, 1])
custom_histogram(axes[2, 1], test['Balance'])
axes[2, 1].set_title('Balance Distribution')

# NumOfProducts
ax_num_products = sns.countplot(data=test, x='NumOfProducts', ax=axes[3, 0])
custom_barplot(ax_num_products, test.shape[0])
axes[3, 0].set_title('NumOfProducts Distribution')

# HasCrCard
ax_has_crcard = sns.countplot(data=test, x='HasCrCard', ax=axes[3, 1])
custom_barplot(ax_has_crcard, test.shape[0])
axes[3, 1].set_title('HasCrCard Distribution')

# IsActiveMember
ax_active_member = sns.countplot(data=test, x='IsActiveMember', ax=axes[4, 0])
custom_barplot(ax_active_member, test.shape[0])
axes[4, 0].set_title('IsActiveMember Distribution')

# Estimated Salary
sns.histplot(test['EstimatedSalary'], bins=30, kde=True, ax=axes[4, 1])
custom_histogram(axes[4, 1], test['EstimatedSalary'])
axes[4, 1].set_title('Estimated Salary Distribution')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Credit Score
sns.boxplot(y=test['CreditScore'], ax=axes[0, 0])
custom_boxplot(axes[0, 0], test['CreditScore'])
axes[0, 0].set_title('Credit Score')

# Age
sns.boxplot(y=test['Age'], ax=axes[0, 1])
custom_boxplot(axes[0, 1], test['Age'])
axes[0, 1].set_title('Age')

# Balance
sns.boxplot(y=test['Balance'], ax=axes[1, 0])
custom_boxplot(axes[1, 0], test['Balance'])
axes[1, 0].set_title('Balance')

# Estimated Salary
sns.boxplot(y=test['EstimatedSalary'], ax=axes[1, 1])
custom_boxplot(axes[1, 1], test['EstimatedSalary'])
axes[1, 1].set_title('Estimated Salary')

plt.tight_layout()
plt.show()

###### test 시각화 해석


- BarPlot, HistPlot 해석
    - **Credit Score(신용점수)**
        - 평균
            - 656.53
        - 분포
            - 대체로 정규분포 형태
            - 신용 점수 분포가 넓게 퍼져있어 다양한 점수 분포
            - 신용점수가 600-700 사이에 가장 많은 고객이 분포

    - **Geography(고객 거주 국가)**
        - 분포
            - France가 전체의 57.4%로 가장 많고, 그 다음 Spain 21.7%, Germany 20.8% 순서

    - **Gender(고객 성별)**
        - 분포
            - 남성이 약 56.3%, 여성이 43.7%

    - **Age(고객 연령)**
        - 평균
            - 38.12
        - 분포
            - 대체로 정규분포 형태
            - 30,40대에 주 고객층이 모여있다
            - 연령이 증가할수록 고객 수가 급격히 감소하는 경향
                - 젊은 고객층(30-40대)에서 고객수가 집중

    - **Tenure(은행 이용 기간)**
        - 분포
            - 모든 기간에 고객수가 고르게 분포되어 있지만, 0년과 10년에 소폭 감소

    - **Balance(계좌잔액)**
        - 평균
            - 55333.61
        - 분포
            - 0인 경우가 가장 많으며, 0을 제외한 나머지 값들은 정규분포에 가깝게 분포
            - 주로 50,000~200,000 구간에 집중

    - **NumOfProducts(고객 이용 은행 상품 수)**
        - 분포
            - 1개 또는 2개 상품을 보유한 고객이 대부분
                - 1개(46.9%), 2개(51.2%)

    - **HasCrCard(신용카드 보유 여부)**
        - 분포
            - 신용카드 보유 고객(1)이 73.3%, 비보유 고객(0)이 24.7%의 분포로 보유한 고객이 더 많다

    - **IsActiveMemebr(활성 회원 여부)**
        - 분포
          - 비활성회원이 50.5%, 활성회원이 49.5%로 차이가 매우 적었다.

    - **EstimatedSalary(추정 연봉)**
        - 평균
            - 112315.15
        - 분포
            - 급여가 0에서 200000까지 고르게 분포
                - 주요 구간 : 50000 - 175000

- BoxPlot
    - CreditScore
        - 이상치
            - O
            - 신용 점수 하단에 이상치 다수 존재
                - 특히 400이하 낮은 점수 고객, 점수가 낮은 일부 고객층 포함
        - 중앙값
            - 660
        - 분포
            - 약간 오른쪽으로 치우친 분포
                - 왼쪽꼬리

    - Age
        - 이상치
            - O
            - 60 이상의 나이에서 이상치 다수 존재
                - 특히 80세 이상 고객이 드물게 포함, 고령층 소수 포함
        - 중앙값
            - 37
        - 분포
            - 오른쪽 긴 꼬리
    - Balance
        - 이상치
            - X
        - 중앙값
            - 0
    - EstimatedSalary
        - 이상치
            - X
        - 중앙값
            - 117,832.23


### 데이터 파악 후 전처리 계획

```
# ID, Surname 제거


# Credit Score => 로그 변환, 스케일링 진행 가능성
- 대체로 정규 분포
- 신용점수 하단에 이상치 다수 존재 -> 왼쪽꼬리
- 필요시 로그 변환 진행
- 분포가 넓게 퍼져있다 -> 필요시 스케일링 진행



# Geography => OneHot Encoding
- 문자형,범주형 변수 : 인코딩 진행
- 고유값 3개
- 순서 X
- OneHot Encoding 진행 가능성

# Gender => Label, Binary Encoding
- 문자형,범주형 변수 : 인코딩 진행
- 고유값 2개 : 라벨 인코딩 or 이진 인코딩

# Age => 로그 변환, 스케일링, 구간 변환 가능성
- 대체로 정규분포 형태
- 60세 이상 이상치 존재 -> 오른쪽 꼬리
- 필요시 로그 변환 진행
- 분포가 넓게 퍼져있다(20대 – 90대) -> 필요시 스케일링 진행
- 분포 30,40대 집중 / 60대 이상은 극소수 -> 이상치(60대 이상)들을 포함시키기 위해 age를 구간으로 사용할 가능성

# Tenure => 그대로 or Label Encoding
- 정수형,범주형 변수 : 그대로 사용 or Label Encoding(일관성 위해 인코딩 진행)
- 0년, 10년만 소폭 감소

# Balance => 로그변환, 스케일링, 이항변수, 클러스터링(범주형) 가능성
- 0인 경우가 매우 많고, 나머지가 정규분포에 가까움 -> 왼쪽 꼬리
- 필요시 로그 변환 혹은 스케일링
- 0 처리 -> 이항변수(범주형)로 만드는 방법
- 범주형(낮음, 중간, 높음) : 클러스터링

# NumOfProducts => OneHot Encoding 가능성
- 정수형, 범주형 변수
- 순서가 없다
- OneHot Encoding 가능성
- 1개(46.9%), 2개(51.1%)가 대부분

# HasCrCard => Label Encoding 가능성
- 정수형, 범주형 변수
- 순서 X
- 이진 변수
-> Label Encoding 적합 가능성(일관성 위해)


# IsActiveMember => Label Encoding 가능성
- 정수형, 범주형 변수
- 순서
- 이진 변수
-> LabelEncoding 적합해보임(일관성 위해)

# EstimatedSalary => 스케일링 가능성
- 분포가 넓게 퍼져있다(0-200000)
- 주요 구간 : 5000 - 175000
-> 스케일링 가능성

# Exited

```

### 1. 전처리 진행
- 다중공선성 확인
- ID, Surname 제거
- 스케일링 X
- 인코딩 O
- 이상치 처리 X
- 중복치 처리 X
- 파생변수 X


#### 변수 제거 및 인코딩

In [ ]:
# train, test 동시 진행

# Id, Surname 제거
train.drop(columns=['id','Surname'],inplace=True)
test.drop(columns=['id','Surname'],inplace=True)

# Geography -> OneHot Encoding
train = pd.get_dummies(train, columns=['Geography'],dtype=int)
test = pd.get_dummies(test, columns=['Geography'], dtype=int)

# Gender -> Label Encoding
encoder = LabelEncoder()
train['Gender'] = encoder.fit_transform(train['Gender'])
test['Gender'] = encoder.transform(test['Gender'])

'''
- 매칭 값 확인 -
코드 :
    original_classes = encoder.classes_
    mapping = {}
    for i, original in enumerate(original_classes):
        mapping[original] = i
    print("매핑:", mapping)

결과 :
    매핑: {'Female': 0, 'Male': 1}
'''

# Tenure -> Label Encoding
encoder = LabelEncoder()
train['Tenure'] = encoder.fit_transform(train['Tenure'])
test['Tenure'] = encoder.transform(test['Tenure'])

'''
- 매칭 값 확인 -
코드 :
    original_classes = encoder.classes_
    mapping = {}
    for i, original in enumerate(original_classes):
        mapping[original] = i
    print("매핑:", mapping)

결과 :
    매핑: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10}
'''

# NumOfProducts -> OneHot Encoding
train = pd.get_dummies(train, columns=['NumOfProducts'],dtype=int)
test = pd.get_dummies(test, columns=['NumOfProducts'], dtype=int)

# HasCrCard -> Label Encoding
encoder = LabelEncoder()
train['HasCrCard'] = encoder.fit_transform(train['HasCrCard'])
test['HasCrCard'] = encoder.transform(test['HasCrCard'])

'''
- 매칭 값 확인 -
코드 :
    original_classes = encoder.classes_
    mapping = {}
    for i, original in enumerate(original_classes):
        mapping[original] = i
    print("매핑:", mapping)

결과 :
    매핑: {0.0: 0, 1.0: 1}
'''

# IsActiveMember -> Label Encoding
encoder = LabelEncoder()
train['IsActiveMember'] = encoder.fit_transform(train['IsActiveMember'])
test['IsActiveMember'] = encoder.transform(test['IsActiveMember'])

'''
- 매칭 값 확인 -
코드 :
    original_classes = encoder.classes_
    mapping = {}
    for i, original in enumerate(original_classes):
        mapping[original] = i
    print("매핑:", mapping)

결과 :
    매핑: {0.0: 0, 1.0: 1}
'''

#### 다중공선성(VIF)

- 인코딩으로 인한 더미변수 제외 모두 다중공선성 X

```
              Feature       VIF
0          CustomerId  1.000260
1         CreditScore  1.000737
2              Gender  1.008577
3                 Age  1.044343
4              Tenure  1.000365
5             Balance  1.696069
6           HasCrCard  1.001405
7      IsActiveMember  1.010977
8     EstimatedSalary  1.000438
9    Geography_France       inf
10  Geography_Germany       inf
11    Geography_Spain       inf
12    NumOfProducts_1       inf
13    NumOfProducts_2       inf
14    NumOfProducts_3       inf
15    NumOfProducts_4       inf
```

In [ ]:
# target 변수 제외: train_features
train_features = train.drop(columns=['Exited'])

# VIF 계산 : (exog : Any, exog_idx : Any ) -> (독립변수, 독립변수 인덱스) -> 독립변수 : 독립 변수들을 포함하는 2D 배열이나 데이터프레임 /
vif_data = pd.DataFrame()
vif_data['Feature'] = train_features.columns
vif_data['VIF'] = [variance_inflation_factor(train_features.values, i) for i in range(train_features.shape[1])]
print(vif_data)

#### 추가 시각화_상관관계 히트맵

- 더미변수끼리의 상관계수 비교 제외
- Exited
    - 양
        - **Age(0.34)**
            - 연령이 높을 수록 이탈율 다소 높다
        - NumOfProducts_1(0.31)
            - 은행 상품수를 1개 보유할 때 이탈율 다소 높다
        - Geography_Germany(0.21)
            - 독일 거주하는 고객이 이탈할 가능성 다소 높다
        - Balacne(0.13)
            - 잔고가 높은 고객이 이탈 가능성이 조금 높다
            - 낮음

    - 음
        - **NumOfProducts_2(-0.38)**
            - 은행 상품수를 2개 보유할 때 이탈율 다소 감소
        - IsActiveMember(-0.21)
            - 활동적인 고객일수록 이탈 가능성이 다소 낮다
        - Gender(-0.15)
            - 남성인 고객이 이탈 가능성이 약간 더 낮다
        - Geography_France(-0.13)
            - 프랑스의 거주하는 고객이 이탈할 가능성 다소 낮다

- 전체
    - 양
        - **Geography_Germany ↔ Balance(0.54)**
            - 독일 거주 고객일수록 잔고가 높은 가능성
        - NumOfProducts_1 ↔ Balance(0.41)
            - 상품을 1개 보유한 고객일수록 잔고가 높을 가능성
        - Age ↔ NumOfProducts_1(0.14)
            - 연령이 높을수록 상품을 1개 보유할 가능성
        - NumOfProducts_1 ↔ Geography_Germany(0.13)
            - 상품을 1개 보유하는 고객이 독일에 거주할 가능성
    - 음
        - **NumOfProducts_2 ↔ Balance (-0.41)**
            - 상품을 2개 보유할수록 잔고가 낮을 가능성
        - Balance ↔ Geography_France (-0.33)
            - 프랑스 거주 고객은 잔고가 낮을 가능성
        - NumOfProducts_2 ↔ Age (-0.17)
            - 상품을 2개 보유한 고객은 나이가 어릴 가능성
        - NumOfProducts_2 ↔ Geography_Germany(-0.15)
            - 상품을 2개 보유한 고객은 독일 거주 가능성이 낮다

In [ ]:
corr = train.corr()

# 히트맵 시각화
plt.figure(figsize=(15, 13))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', center=0, cbar=True)

# 그래프 제목 추가
plt.title('Correlation Heatmap')

# 그래프 출력
plt.show()

#### 클러스터링

In [ ]:
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

# 상관관계 계산
corr = train.corr()

# 상관관계 클러스터링
sns.clustermap(corr, method='ward', cmap='coolwarm', annot=True, fmt=".2f",
               figsize=(10, 10), dendrogram_ratio=(.1, .2))
plt.title('Correlation Clustering')
plt.show()

## 2.모델링

### 데이터 분리

In [ ]:
# feature, target 분리
train_feature = train.drop(columns='Exited')
train_target = train['Exited']

# 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(train_feature, train_target,
                                                      test_size = 0.3,
                                                      random_state=0)

# 분리 데이터 크기 확인
for i in [X_train, X_valid, y_train, y_valid] :
  print(i.shape)

### AutoML
- gbc : Gradient Boosting Classifier
- lightgbm : Light Gradient Boosting Machine
- catboost : CatBoost Classifier
- xgboost : Extreme Gradient Boosting
- ada : Ada Boost Classifier

In [ ]:
# 인덱스 번호 새로 할당
X_train_automl = X_train.reset_index(drop=True)
y_train_automl = y_train.reset_index(drop=True)

# X_train, y_train 합치기
Xy_train_automl = pd.concat([X_train_automl, y_train_automl], axis=1)

# AutoML 모델 세팅
clf = setup(data=Xy_train_automl,
            target = 'Exited',
            train_size = 0.7,
            data_split_shuffle=True,
            session_id = 0,
            fold = 5
            )

# AutoML top5 model 설정
top5_model = compare_models(fold=5, round=3, n_select=5, sort = 'AUC', errors='ignore',verbose=True)
top5_model

### Optuna
- gbc
```
Best AUC: 0.8879903418757644
Best hyperparameters:
  n_estimators: 441
  learning_rate: 0.045696033561884446
  max_depth: 7
  min_samples_split: 2
  min_samples_leaf: 2
  subsample: 0.9753636860597981
  max_features: log2
  loss: exponential
  ccp_alpha: 2.8658360294722587e-05
  validation_fraction: 0.27267380097000604
  n_iter_no_change: 17
  tol: 0.005721221703263734
  min_impurity_decrease: 0.04492762681125898
  max_leaf_nodes: 65
```
- lightgbm
```
Best AUC: 0.8893180320851476
Best hyperparameters:
  num_boost_round: 424
  learning_rate: 0.04631415040823912
  num_leaves: 38
  max_depth: 9
  min_data_in_leaf: 47
  feature_fraction: 0.9024601968835392
  bagging_fraction: 0.7342793249627353
  bagging_freq: 2
  min_gain_to_split: 0.3017007245252093
  lambda_l1: 0.07886266683723922
  lambda_l2: 0.0859776680854032
  tree_learner: serial
  max_bin: 310
  early_stopping_rounds: 38
  num_threads: 3
  scale_pos_weight: 3.8048327134060944
```
- CatBoost
```
Best AUC: 0.8894975131185816
Best hyperparameters:
  iterations: 622
  learning_rate: 0.15620856452750326
  depth: 3
  l2_leaf_reg: 0.023092783762797234
  random_strength: 0.013271842224732251
  bagging_temperature: 6.81082830068841
  grow_policy: SymmetricTree
  border_count: 117
  od_wait: 15
```
- xgboost
```
Best AUC: 0.8855466532778464
Best hyperparameters:
  num_round: 173
  alpha: 0.8296136024447837
  base_score: 0.549188827783536
  booster: gbtree
  colsample_bylevel: 0.6341674144934313
  colsample_bynode: 0.7675482440337507
  colsample_bytree: 0.8777999787302608
  eta: 0.28259482675888387
  eval_metric: auc
  gamma: 0.2819080182819892
  grow_policy: depthwise
  lambda: 3.426745960231184
  max_bin: 340
  max_delta_step: 10
  max_depth: 9
  max_leaves: 33
  min_child_weight: 4.375849619589128
  objective: binary:logistic
  scale_pos_weight: 7.596583814614301
  seed: 55
  subsample: 0.9365805881615846
  verbosity: 2
  early_stopping_rounds: 54
```
- ada
```
Best AUC: 0.887546021751948
Best hyperparameters:
  n_estimators: 194
  learning_rate: 0.07457707279949315
  algorithm: SAMME.R
  random_state: 641
  max_depth: 4
  min_samples_split: 9
  min_samples_leaf: 2
  max_features: None
  max_leaf_nodes: 52
  min_impurity_decrease: 0.0003868616444805657
```


#### GBC : Gradient Boosting Classifier

In [ ]:
sampler = TPESampler(seed=0)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),                          # 트리의 개수
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),                     # 학습률
        'max_depth': trial.suggest_int('max_depth', 3, 10),                                   # 트리의 최대 깊이
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),                   # 분할을 위한 최소 샘플 수
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),                     # 리프 노드에 최소한 있어야 할 샘플 수
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),                              # 샘플링 비율
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),    # 분할에 사용할 특성의 수
        'loss': trial.suggest_categorical('loss', ['log_loss', 'exponential']),               # 손실 함수
        'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.1),                              # 가지치기 파라미터
        'random_state': 0,                                                                    # 고정된 랜덤 시드
        'validation_fraction': trial.suggest_float('validation_fraction', 0.1, 0.3),          # 검증 데이터 비율
        'n_iter_no_change': trial.suggest_int('n_iter_no_change', 5, 20),                     # 성능 향상이 없을 경우 훈련 중단
        'tol': trial.suggest_float('tol', 1e-4, 1e-2),                                        # 수렴을 위한 tolerance 값
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.1),      # 불순도 감소 기준
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 100),                       # 최대 리프 노드 수
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    auc_list = []

    for _, (train_index, valid_index) in enumerate(cv.split(train_feature,train_target)):
        X_train, y_train = train_feature.iloc[train_index], train_target.iloc[train_index]
        X_valid, y_valid = train_feature.iloc[valid_index], train_target.iloc[valid_index]


        gbc_model = GradientBoostingClassifier(**params)
        gbc_model.fit(X_train, y_train)

        # 예측 확률 계산
        y_prob = gbc_model.predict_proba(X_valid)[:,1]
        auc_score = roc_auc_score(y_valid, y_prob)
        auc_list.append(auc_score)

    # 평균 auc 반환
    mean_auc = np.mean(auc_list)
    return mean_auc

# optuna 최적화 실행(AUC 최대화)
optuna_gbc = optuna.create_study(direction='maximize', sampler=sampler)
optuna_gbc.optimize(objective, n_trials=50)

# 최적 결과 출력
best_trial = optuna_gbc.best_trial
print(f"Best AUC: {best_trial.value}")
print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

#### LightGBM : Light Gradient Boosting Machine

In [ ]:
sampler = TPESampler(seed=0)

def objective(trial):
    params = {
        'num_boost_round': trial.suggest_int('num_boost_round', 100, 1000),                                   # 최대 부스팅 반복 횟수 (트리의 수) / default: 100
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),                                     # 학습률 (각 예제마다 가중치를 업데이트하는 비율) / default: 0.1
        'num_leaves': trial.suggest_int('num_leaves', 31, 128),                                               # 트리의 최대 잎 수 / default: 64
        'max_depth': trial.suggest_int('max_depth', -1, 15),                                                  # 트리의 최대 깊이 / default: 6
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),                                   # 각 리프 노드에 최소 데이터 수 / default: 3
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),                                # 각 트리에서 선택할 특징의 비율 / default: 0.9
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),                                # 배깅의 비율 (데이터의 일부를 무작위로 선택) / default: 0.9
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),                                             # 배깅을 수행하는 빈도 / default: 1
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),                              # 각 트리에서 사용할 최소 게인 / default: 0.0
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 0.1),                                              # L1 정규화 / default: 0.0
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 0.1),                                              # L2 정규화 / default: 0.0
        'tree_learner': trial.suggest_categorical('tree_learner', ['serial', 'feature', 'data', 'voting']),   # 트리 훈련자 유형 / default: 'serial'
        'max_bin': trial.suggest_int('max_bin', 255, 512),                                                    # 최대 빈 수 (기능 값을 버킷화하는 데 사  용) / default: 255
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 50),                          # 조기 중단 라운드 / default: 10
        'metric': 'auc',                                                                                      # 평가 지표 (auc 고정)
        'num_threads': trial.suggest_int('num_threads', 1, 8),                                                # 트리 학습에 사용하는 스레드 수 / default: 0
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 5.0),                                # 스케일링된 레이블의 가중치 (바이너리 분류에만 사용) / default: 1.0
        'verbosity': 0                                                                                        # 인쇄메시지
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    auc_list = []

    for _, (train_index, valid_index) in enumerate(cv.split(train_feature,train_target)):
        X_train, y_train = train_feature.iloc[train_index], train_target.iloc[train_index]
        X_valid, y_valid = train_feature.iloc[valid_index], train_target.iloc[valid_index]

        # LightGBM 모델
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)

        # lightGBM 모델 학습
        lgb_model = lgb.train(
            params,
            train_data,
            valid_sets = [train_data, valid_data]
        )

        # 예측 확률 계산
        y_prob = lgb_model.predict(X_valid)
        auc_score = roc_auc_score(y_valid, y_prob)
        auc_list.append(auc_score)

    # 평균 auc 반환
    mean_auc = np.mean(auc_list)
    return mean_auc

# optuna 최적화 실행(AUC 최대화)
optuna_lgb = optuna.create_study(direction='maximize', sampler=sampler)
optuna_lgb.optimize(objective, n_trials=50)

# 최적 결과 출력
best_trial = optuna_lgb.best_trial
print(f"Best AUC: {best_trial.value}")
print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

#### CatBoost : CatBoost Classifier

In [ ]:
sampler = TPESampler(seed=0)

def objective(trial):
    # 하이퍼파라미터 정의
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),                                               # 트리 개수 (Boosting 반복 횟수) / default: 1000
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 1.0),                                  # 학습률 (트리의 가중치를 조정하는 비율) / default: 0.03
        "depth": trial.suggest_int("depth", 1, 16),                                                             # 트리의 최대 깊이 (과적합 방지) / default: 6
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 10.0),                                     # L2 정규화 계수 (과적합 방지) / default: 3.0
        "random_strength": trial.suggest_loguniform("random_strength", 1e-2, 10.0),                             # 트리 분할 시 무작위성 정도 (과적합 방지) / default: 1.0
        "bagging_temperature": trial.suggest_loguniform("bagging_temperature", 1e-5, 10.0),                     # 베이지안 부트스트랩 샘플링 강도 (1 이상이면 샘플링 강화) / default: 1.0
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),   # 트리 성장 정책 (균형 성장 vs 깊이 우선 vs 손실 기반) / default: SymmetricTree
        "border_count": trial.suggest_int("border_count", 1, 255),                                              # 연속형 피처를 구간화할 때 사용할 분할 개수 / default: 254
        "thread_count": -1,                                                                                     # 사용할 CPU 스레드 개수 (-1이면 모든 코어 사용) / default: -1
        "random_seed": 42,                                                                                      # 재현 가능성을 위한 난수 시드 고정 / default: 없음
        "eval_metric": "AUC",                                                                                   # 평가 지표 (Area Under Curve) / default: Logloss
        "verbose": 0,                                                                                           # 학습 과정 출력 여부 (0이면 출력 안 함)
        "od_type": "Iter",                                                                                      # 조기 종료 조건 (Iteration 기반) / default: IncToDec
        "od_wait": trial.suggest_int("od_wait", 10, 50),                                                        # 조기 종료를 위한 대기 스텝 수 / default: 50
        "task_type": "CPU",                                                                                     # 실행 환경 설정 (CPU 사용)
        "loss_function": "Logloss"                                                                              # 손실 함수 (이진 분류 문제에서는 Logloss 사용)
    }


    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    auc_list = []

    for _, (train_index, valid_index) in enumerate(cv.split(train_feature,train_target)):
        X_train, y_train = train_feature.iloc[train_index], train_target.iloc[train_index]
        X_valid, y_valid = train_feature.iloc[valid_index], train_target.iloc[valid_index]


        cat_model = CatBoostClassifier(**params)
        cat_model.fit(X_train, y_train,
                      eval_set = (X_valid, y_valid),early_stopping_rounds = 50, verbose = 0)

        # 예측 확률 계산
        y_score = cat_model.predict_proba(X_valid)[:,1]
        auc_score = roc_auc_score(y_valid, y_score)
        auc_list.append(auc_score)

    # 평균 auc 반환
    mean_auc = np.mean(auc_list)
    return mean_auc

# optuna 최적화 실행(AUC 최대화)
optuna_cat = optuna.create_study(direction='maximize', sampler=sampler)
optuna_cat.optimize(objective, n_trials=50)

# 최적 결과 출력
best_trial = optuna_cat.best_trial
print(f"Best AUC: {best_trial.value}")
print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

#### XGBoost : Extreme Gradient Boosting

In [ ]:
sampler = TPESampler(seed=0)

def objective(trial):
    params = {
        'num_round': trial.suggest_int('num_round', 100, 1000),                               # 부스팅 반복 횟수 (트리 개수) / default: 100
        'alpha': trial.suggest_float('alpha', 0.0, 1.0),                                      # L1 정규화 항 (Lasso 규제, 가중치 감소 효과) / default: 0.0
        'base_score': trial.suggest_float('base_score', 0.0, 1.0),                            # 초기 예측값 (모든 샘플의 기본 예측값) / default: 0.5
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear']),              # 부스팅 방식 ('gbtree': 트리 기반, 'gblinear': 선형 모델) / default: 'gbtree'
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),              # 트리의 각 레벨에서 선택할 피처 비율 / default: 1.0
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),                # 트리 각 노드에서 선택할 피처 비율 / default: 1.0
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),                # 각 트리에서 선택할 피처 비율 (컬럼 샘플링) / default: 1.0
        'eta': trial.suggest_float('eta', 0.01, 0.3),                                         # 학습률 (트리 가중치 업데이트 비율) / default: 0.3
        'eval_metric': 'auc',                                                                 # 평가 지표 (AUC로 고정)
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),                                      # 트리 분할을 위한 최소 손실 감소값 (클수록 보수적 분할) / default: 0.0
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),  # 트리 성장 방식 ('depthwise': 깊이 우선, 'lossguide': 손실 기반) / default: 'depthwise'
        'lambda': trial.suggest_float('lambda', 0.0, 10.0),                                   # L2 정규화 항 (Ridge 규제, 과적합 방지) / default: 1.0
        'max_bin': trial.suggest_int('max_bin', 10, 512),                                     # 연속형 변수를 이산화할 때 사용할 빈 개수 / default: 256
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),                         # 클래스 불균형 보정 (가중치 변화 제한) / default: 0
        'max_depth': trial.suggest_int('max_depth', 3, 12),                                   # 개별 트리의 최대 깊이 (깊을수록 모델 복잡도 증가) / default: 6
        'max_leaves': trial.suggest_int('max_leaves', 0, 50),                                 # 트리의 최대 리프 노드 개수 (0이면 제한 없음) / default: 0
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),               # 리프 노드가 분할되기 위한 최소 가중치 합 (클수록 덜 분할) / default: 1.0
        'objective': 'binary:logistic',                                                       # 학습 목표 (이진 분류: 로지스틱 회귀) / default: 'binary:logistic'
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 10.0),               # 불균형 데이터 조정 가중치 / default: 1.0
        'seed': trial.suggest_int('seed', 0, 1000),                                           # 난수 시드 (재현 가능성 확보) / default: 0
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),                              # 트리 학습 시 사용하는 샘플링 비율 (과적합 방지) / default: 1.0
        'verbosity': trial.suggest_int('verbosity', 0, 3),                                    # 출력 메시지 수준 (0: 없음, 3: 상세 로그) / default: 1
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    auc_list = []

    for _, (train_index, valid_index) in enumerate(cv.split(train_feature,train_target)):
        X_train, y_train = train_feature.iloc[train_index], train_target.iloc[train_index]
        X_valid, y_valid = train_feature.iloc[valid_index], train_target.iloc[valid_index]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)

        xgb_model = xgb.train(
            params,
            dtrain,
            evals=[(dtrain, 'train'), (dvalid, 'valid')],
            early_stopping_rounds=trial.suggest_int('early_stopping_rounds', 10, 100)
        )

        # 예측 확률 계산
        y_prob = xgb_model.predict(dvalid)
        auc_score = roc_auc_score(y_valid, y_prob)
        auc_list.append(auc_score)

    # 평균 auc 반환
    mean_auc = np.mean(auc_list)
    return mean_auc

# optuna 최적화 실행(AUC 최대화)
optuna_xgb = optuna.create_study(direction='maximize', sampler=sampler)
optuna_xgb.optimize(objective, n_trials=50)

# 최적 결과 출력
best_trial = optuna_xgb.best_trial
print(f"Best AUC: {best_trial.value}")
print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

#### AdaBoost : Ada Boost Classifier

In [ ]:
sampler = TPESampler(seed=0)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 500),                         # 부스팅할 약한 학습기의 개수 / default: 50
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),         # 학습률 (작을수록 보수적 학습) / default: 1.0
        'algorithm': trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R']),          # 부스팅 알고리즘 ('SAMME': 분류기 확률 사용 안 함, 'SAMME.R': 확률 사용) / default: 'SAMME.R'
        'random_state': trial.suggest_int('random_state', 0, 1000)                          # 난수 시드 (재현 가능성 확보) / default: None
    }

    # 기본 학습기 (약한 학습기)로 결정 트리 사용 시 세부 하이퍼파라미터 설정
    base_estimator_params = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),                                 # 트리의 최대 깊이 (깊을수록 복잡한 모델) / default: None (무제한)
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),                 # 노드를 분할하기 위한 최소 샘플 개수 / default: 2
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),                   # 리프 노드에 있어야 하는 최소 샘플 개수 / default: 1
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),  # 트리 분할 시 사용할 최대 특성 개수 / default: None (모든 특성 사용)
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 100, log=True),           # 리프 노드 최대 개수 (과적합 방지) / default: None (제한 없음)
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.1)     # 최소 불순도 감소 기준 (값이 클수록 덜 분할) / default: 0.0
    }


    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    auc_list = []

    for _, (train_index, valid_index) in enumerate(cv.split(train_feature,train_target)):
        X_train, y_train = train_feature.iloc[train_index], train_target.iloc[train_index]
        X_valid, y_valid = train_feature.iloc[valid_index], train_target.iloc[valid_index]


        base_estimator = DecisionTreeClassifier(**base_estimator_params)


        ada_model = AdaBoostClassifier(estimator=  base_estimator, **params)
        ada_model.fit(X_train,y_train)

        # 예측 확률 계산
        y_prob = ada_model.predict_proba(X_valid)[:,1]
        auc_score = roc_auc_score(y_valid, y_prob)
        auc_list.append(auc_score)

    # 평균 auc 반환
    mean_auc = np.mean(auc_list)
    return mean_auc

# optuna 최적화 실행(AUC 최대화)
optuna_ada = optuna.create_study(direction='maximize', sampler=sampler)
optuna_ada.optimize(objective, n_trials=50)

# 최적 결과 출력
best_trial = optuna_ada.best_trial
print(f"Best AUC: {best_trial.value}")
print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

### 보팅 : 서로 다른 알고리즘을 가진 분류기 결합

- 하드 보팅
    - 예측값들 중 다수 분류기가 결정한 예측값을 최종 보팅 결괏값(다수결)
- **소프트 보팅**
    - **레이블 값 결정 확률을 모두 더하고 이를 평균해서 이들 중 확률이 가장 높은 레이블 값**
        - gbc: 0.8879903418757644
        - lightgbm: 0.8893180320851476
        - catboost: 0.8894975131185816(가장 높은 성능)
        - xgboost: 0.8855466532778464
        - ada: 0.887546021751948
        

#### 조합 생성

In [ ]:
models = ['catboost','lightgbm','gbc','ada','xgboost']

model_combinations = []
for i in range(1,len(models) + 1) :
  model_combinations.extend(combinations(models,i))


for combo in model_combinations :
  print(combo)

#### 가중치

In [ ]:
roc_score = {
    'catboost' :  0.8894975131185816,
    'lightgbm' : 0.8893180320851476,
    'gbc' : 0.8879903418757644,
    'ada' : 0.887546021751948,
    'xgboost' : 0.8855466532778464
}

total_roc = sum(roc_score.values())
weights = {model : roc_score[model] / total_roc for model in roc_score}

#### 소프트 보팅 구현

In [ ]:
# 각 모델의 최적 하이퍼파라미터로 초기화된 모델
models_dict = {
    'catboost': CatBoostClassifier(
        iterations=622,
        learning_rate=0.15620856452750326,
        depth=3,
        l2_leaf_reg=0.023092783762797234,
        random_strength=0.013271842224732251,
        bagging_temperature=6.81082830068841,
        grow_policy='SymmetricTree',
        border_count=117,
        od_wait=15,
        verbose=0,  # 학습 중 출력 비활성화
        random_state=42
    ),
    'lightgbm': LGBMClassifier(
        n_estimators=424,  # num_boost_round 대신 n_estimators로 변경
        learning_rate=0.04631415040823912,
        num_leaves=38,
        max_depth=9,
        min_child_samples=47,
        feature_fraction=0.9024601968835392,
        bagging_fraction=0.7342793249627353,
        bagging_freq=2,
        min_split_gain=0.3017007245252093,
        reg_alpha=0.07886266683723922,
        reg_lambda=0.0859776680854032,
        tree_learner='serial',
        max_bin=310,
        num_threads=3,
        scale_pos_weight=3.8048327134060944
    ),
    'gbc': GradientBoostingClassifier(
        n_estimators=441,
        learning_rate=0.045696033561884446,
        max_depth=7,
        min_samples_split=2,
        min_samples_leaf=2,
        subsample=0.9753636860597981,
        max_features='log2',
        loss='exponential',
        ccp_alpha=2.8658360294722587e-05,
        validation_fraction=0.27267380097000604,
        n_iter_no_change=17,
        tol=0.005721221703263734,
        min_impurity_decrease=0.04492762681125898,
        max_leaf_nodes=65
    ),
    'ada': AdaBoostClassifier(
        estimator=DecisionTreeClassifier(
            max_depth=4,
            min_samples_split=9,
            min_samples_leaf=2,
            max_features=None,
            max_leaf_nodes=52,
            min_impurity_decrease=0.0003868616444805657,
            random_state=641
        ),
        n_estimators=194,
        learning_rate=0.07457707279949315,
        algorithm='SAMME.R',
        random_state=641
    ),
    'xgboost': XGBClassifier(
        n_estimators=173,  # num_round 대신 n_estimators로 변경
        alpha=0.8296136024447837,
        base_score=0.549188827783536,
        booster='gbtree',
        colsample_bylevel=0.6341674144934313,
        colsample_bynode=0.7675482440337507,
        colsample_bytree=0.8777999787302608,
        learning_rate=0.28259482675888387,
        eval_metric='auc',
        gamma=0.2819080182819892,
        grow_policy='depthwise',
        reg_lambda=3.426745960231184,
        max_bin=340,
        max_delta_step=10,
        max_depth=9,
        max_leaves=33,
        min_child_weight=4.375849619589128,
        objective='binary:logistic',
        scale_pos_weight=7.596583814614301,
        random_state=55,
        subsample=0.9365805881615846,
        verbosity=2
    )
}

# 소프트 보팅 성능 평가
best_combo = None
best_auc = 0
results = []

# 교차검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for combo in model_combinations:
    estimators = [(name, models_dict[name]) for name in combo]
    weights_for_combo = [weights[name] for name in combo]

    voting_clf = VotingClassifier(
        estimators=estimators,
        voting='soft',
        weights=weights_for_combo
    )

    roc_list = []  # 교차검증 roc 기록

    # 교차 검증 루프
    for train_idx, valid_idx in cv.split(train_feature, train_target):
        X_train, y_train = train_feature.iloc[train_idx], train_target.iloc[train_idx]
        X_valid, y_valid = train_feature.iloc[valid_idx], train_target.iloc[valid_idx]

        # 모델 학습
        voting_clf.fit(X_train, y_train)

        # 예측 및 ROC 계산
        y_prob = voting_clf.predict_proba(X_valid)[:, 1]
        roc = roc_auc_score(y_valid, y_prob)
        roc_list.append(roc)

    # 교차 검증 평균 AUC 계산
    mean_roc = sum(roc_list) / len(roc_list)

    # 결과 저장
    results.append({
        "combination": combo,
        "mean_roc": mean_roc,
        "roc_per_fold": roc_list,  # 폴드별 점수 추가
        "weights": weights_for_combo
    })

# 결과 정렬 (ROC 기준 내림차순)
results = sorted(results, key=lambda x: x["mean_roc"], reverse=True)

# 모든 결과 출력
print("=" * 50)
print("All Voting Model Results:")
print("=" * 50)
for result in results:
    print(f"Combination: {result['combination']}, Mean ROC: {result['mean_roc']:.4f}, "
          f"Fold AUCs: {result['roc_per_fold']}, Weights: {result['weights']}")

# 최적의 결과 출력
best_result = results[0]           #  가장 높은 점수
print("=" * 50)
print(f"Best Model Combination: {best_result['combination']}")
print(f"Best AUC: {best_result['mean_roc']:.4f}")
print(f"Fold AUCs: {best_result['roc_per_fold']}")
print(f"Best Weights: {best_result['weights']}")
print("=" * 50)

## 3.최종 모델

```
Best Model Combination: ('catboost', 'lightgbm', 'gbc')
Best AUC: 0.8900
Fold AUCs: [0.8900562631197824, 0.8898752206337519, 0.8913648217131387, 0.8912097065381658, 0.8874405758789176]
Best Weights: [0.20034185481390882, 0.20030143023417502, 0.2000023940758461]
```

### 데이터 분리

In [ ]:
# feature, target 분리
train_feature = train.drop(columns='Exited')
train_target = train['Exited']

# 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(train_feature, train_target,
                                                      test_size = 0.3,
                                                      random_state=0)

### 모델 생성

```
<최종점수>
Final Model AUC Score: 0.8903
```

In [ ]:
final_models = {
    'lightgbm': LGBMClassifier(
        n_estimators=424,
        learning_rate=0.04631415040823912,
        num_leaves=38,
        max_depth=9,
        min_child_samples=47,
        feature_fraction=0.9024601968835392,
        bagging_fraction=0.7342793249627353,
        bagging_freq=2,
        min_split_gain=0.3017007245252093,
        reg_alpha=0.07886266683723922,
        reg_lambda=0.0859776680854032,
        tree_learner='serial',
        max_bin=310,
        num_threads=3,
        scale_pos_weight=3.8048327134060944
    ),
    'catboost': CatBoostClassifier(
        iterations=622,
        learning_rate=0.15620856452750326,
        depth=3,
        l2_leaf_reg=0.023092783762797234,
        random_strength=0.013271842224732251,
        bagging_temperature=6.81082830068841,
        grow_policy='SymmetricTree',
        border_count=117,
        od_wait=15,
        verbose=0,
        random_state=42
    ),
    'gbc': GradientBoostingClassifier(
        n_estimators=441,
        learning_rate=0.045696033561884446,
        max_depth=7,
        min_samples_split=2,
        min_samples_leaf=2,
        subsample=0.9753636860597981,
        max_features='log2',
        loss='exponential',
        ccp_alpha=2.8658360294722587e-05,
        validation_fraction=0.27267380097000604,
        n_iter_no_change=17,
        tol=0.005721221703263734,
        min_impurity_decrease=0.04492762681125898,
        max_leaf_nodes=65
    )

}

# 소프트 보팅 모델 생성
final_voting_model = VotingClassifier(
    estimators=[('lightgbm', final_models['lightgbm']), ('catboost', final_models['catboost']),('gbc', final_models['gbc'])],
    voting='soft',
    weights=[0.20034185481390882, 0.20030143023417502, 0.2000023940758461]  # 최적 가중치 적용
)

# 학습
final_voting_model.fit(X_train, y_train)

# 예측
y_prob = final_voting_model.predict_proba(X_valid)[:, 1]
auc_score = roc_auc_score(y_valid, y_prob)

# 결과 출력
print(f"Final Model AUC Score: {auc_score:.4f}")

### 제출 파일 생성

In [ ]:
# 테스트 데이터 예측
test_prob = final_voting_model.predict_proba(test)[:, 1]

submission['Exited'] = test_prob
submission.to_csv('origin_submission.csv',index=False)

### 변수 중요도

In [ ]:
import numpy as np
feature_importance = np.array([0.1, 0.3, 0.05, 0.4, 0.15])

sorted_indices = np.argsort(feature_importance)  # 0.05(2번 인덱스), 0.1(0번 인덱스), 0.15(4번 인덱스), 0.3(1번 인덱스), 0.4(3번 인덱스)
print(sorted_indices)   # [2 0 4 1 3]
# 출력: [2 0 4 1 3] (작은 값부터 정렬된 인덱스)

# 내림차순으로 정렬된 인덱스 -> 큰것부터 작은 것으로
sorted_indices_desc = np.argsort(feature_importance)[::-1]
print(sorted_indices_desc)  # [3 1 4 0 2]

In [ ]:
# 1. Feature Importance 계산 및 시각화(각 모델 feature importance 평균 사용)

def plot_feature_importance(model, model_columns):
    feature_importance = np.zeros(len(model_columns))
    for name, estimator in final_voting_model.named_estimators_.items():                            # named_estimators_ : 개별 추정기(estimator)들을 딕셔너리 형태로 저장
        if hasattr(estimator, "feature_importances_"):                                              # hasattr(객체, "속성_이름") : 객체가 속성을 갖고 있으면 True, 아니면 False 반환
            feature_importance += estimator.feature_importances_                                    # 각각의 모델 변수 중요도 더하기
    feature_importance /= len(final_voting_model.named_estimators_)                                 # 모델 개수로 나누기

    # Feature 중요도 시각화
    sorted_idx = np.argsort(feature_importance)[::-1]                                               # np.argsort() : 정렬된 값의 인덱스 반환 [start:stop:step] / [:: -1] -> 내림차순으로 변경
    sorted_features = np.array(model_columns)[sorted_idx]                                           # 변수 중요도에 따라 변수 명 정렬(인덱스따라)
    sorted_importances = feature_importance[sorted_idx]                                             # 변수 중요도에 따라 값 정렬

    plt.figure(figsize=(10, 8))
    bars = plt.barh(range(len(model_columns)), sorted_importances[::-1], align="center")            # barh : 가로 막대그래프 생성(기본적으로 y값 작은것부터 위로 쌓임) / sorted_importances[::-1] : 막대 길이 조정 / align : 막대 중심 위치
    plt.yticks(range(len(model_columns)), sorted_features[::-1])                                    # sorted_features[::-1] 사용 이유 : 위와 동일. 기본적으로 작은것부터 위로 쌓임
    plt.xlabel("Feature Importance")
    plt.title("Feature Importance from Model")

    # 막대에 값 표시
    for bar, value in zip(bars, sorted_importances[::-1]):
        plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{value:.2f}', va='center')    # plt.text(x, y, s, va='center') / bar.get_width() : 텍스트 x좌표, bar.get_y() + bar.get_height()/2 : 텍스트 y좌표, f'{value:.2f}', va='center' : 표시 할 텍스트

    plt.show()

# 2. Permutation Importance 계산 및 시각화
def plot_permutation_importance(model, X_valid, y_valid, feature_names):
    perm_importance = permutation_importance(
        model, X_valid, y_valid, scoring='roc_auc', n_repeats=5, random_state=42
    )

    # Feature 중요도 정렬
    sorted_idx = np.argsort(perm_importance.importances_mean)[::-1]                                 # 중요도 높은 순서로 정렬
    sorted_features = np.array(feature_names)[sorted_idx]                                           # 정렬된 변수명
    sorted_importances = perm_importance.importances_mean[sorted_idx]                               # 정렬된 변수 중요도

    # Feature 중요도 시각화
    plt.figure(figsize=(10, 8))
    bars = plt.barh(range(len(feature_names)), sorted_importances[::-1], align="center")            # 내림차순 정렬한 막대그래프
    plt.yticks(range(len(feature_names)), sorted_features[::-1])                                    # y축 변수명도 동일한 순서로 정렬
    plt.xlabel("Permutation Importance")
    plt.title("Permutation Importance from Validation Set")

    # 막대에 값 표시
    for bar, value in zip(bars, sorted_importances[::-1]):
        plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{value:.4f}', va='center')

    plt.show()

# Feature Importance 그래프 생성
plot_feature_importance(final_voting_model, train_feature.columns)

# Permutation Importance 그래프 생성
plot_permutation_importance(final_voting_model, X_valid, y_valid, train_feature.columns)


#### feature importance


In [ ]:
from IPython.display import Image

Image('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/원본_feature_importance.png')

#### permutation importance

In [ ]:
Image('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/원본_permutation_importance.png')

# 최종 모델 실행 코드

- 성능 : Final Model AUC Score: 0.8902

In [ ]:
# 기본 패키지
import pandas as pd  # 데이터프레임 생성 및 조작
import numpy as np  # 수치 계산 및 배열 처리

# 시각화
import matplotlib.pyplot as plt  # 기본적인 시각화 도구
import seaborn as sns  # 고급 시각화 도구 (matplotlib 기반)
import scipy.cluster.hierarchy as sch # 클러스터링


# 데이터 전처리
from sklearn.preprocessing import LabelEncoder  # 레이블 인코딩
# pd.get_dummies로 OneHotEncoder 대체 가능

# 데이터 분할
from sklearn.model_selection import train_test_split  # 데이터 분리
from sklearn.model_selection import StratifiedKFold  # 층화 교차 검증

# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor  # 분산팽창계수(VIF) 계산

# 계층적 클러스터링
import scipy.cluster.hierarchy as sch  # 계층적 군집화 알고리즘

# Optuna: 하이퍼파라미터 최적화
import optuna  # 최적화 도구
from optuna.samplers import TPESampler  # 샘플링 알고리즘 (TPE)

# 머신러닝 모델들
from sklearn.ensemble import GradientBoostingClassifier  # Gradient Boosting 모델
from sklearn.ensemble import RandomForestClassifier  # Random Forest 모델
from sklearn.ensemble import AdaBoostClassifier  # AdaBoost 모델
from sklearn.tree import DecisionTreeClassifier  # 의사결정 트리 모델

# LightGBM
from lightgbm import LGBMClassifier  # LightGBM 분류기
import lightgbm as lgb  # LightGBM 라이브러리

# XGBoost
from xgboost import XGBClassifier  # XGBoost 분류기
import xgboost as xgb  # XGBoost 라이브러리

# CatBoost
from catboost import CatBoostClassifier, Pool  # CatBoost 분류기 및 데이터 준비 객체
from catboost.utils import eval_metric  # CatBoost 평가 지표 계산

# 성능 평가 지표
from sklearn.metrics import roc_auc_score  # ROC AUC 점수 계산

# 모델 앙상블
from sklearn.ensemble import VotingClassifier  # 앙상블 학습을 위한 VotingClassifier

# 기타 도구
from itertools import combinations  # 조합 생성
from sklearn.inspection import permutation_importance  # 변수 중요도 계산 (Permutation Importance)

# 출력 관련
from IPython.display import Image  # Jupyter Notebook에서 이미지 표시

# PyCaret: AutoML (자동화된 머신러닝 워크플로우)
from pycaret.classification import *

# 설정
pd.set_option('display.max_columns', None)  # 최대 컬럼 설정

In [ ]:
# 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/train.csv')
test = pd.read_csv('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/파이널 프로젝트_이정수/분류/sample_submission.csv')

# 변수 제거 및 인코딩
# train, test 동시 진행

# Id, Surname 제거
train.drop(columns=['id','Surname'],inplace=True)
test.drop(columns=['id','Surname'],inplace=True)

# Geography -> OneHot Encoding
train = pd.get_dummies(train, columns=['Geography'],dtype=int)
test = pd.get_dummies(test, columns=['Geography'], dtype=int)

# Gender -> Label Encoding
encoder = LabelEncoder()
train['Gender'] = encoder.fit_transform(train['Gender'])
test['Gender'] = encoder.transform(test['Gender'])

# Tenure -> Label Encoding
encoder = LabelEncoder()
train['Tenure'] = encoder.fit_transform(train['Tenure'])
test['Tenure'] = encoder.transform(test['Tenure'])


# NumOfProducts -> OneHot Encoding
train = pd.get_dummies(train, columns=['NumOfProducts'],dtype=int)
test = pd.get_dummies(test, columns=['NumOfProducts'], dtype=int)

# HasCrCard -> Label Encoding
encoder = LabelEncoder()
train['HasCrCard'] = encoder.fit_transform(train['HasCrCard'])
test['HasCrCard'] = encoder.transform(test['HasCrCard'])

# IsActiveMember -> Label Encoding
encoder = LabelEncoder()
train['IsActiveMember'] = encoder.fit_transform(train['IsActiveMember'])
test['IsActiveMember'] = encoder.transform(test['IsActiveMember'])

# feature, target 분리
train_feature = train.drop(columns='Exited')
train_target = train['Exited']

# 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(train_feature, train_target,
                                                      test_size = 0.3,
                                                      random_state=0)

# 최종 모델 생성 : final_models
final_models = {
    'lightgbm': LGBMClassifier(
        n_estimators=424,  # num_boost_round 대신 n_estimators로 변경
        learning_rate=0.04631415040823912,
        num_leaves=38,
        max_depth=9,
        min_child_samples=47,
        feature_fraction=0.9024601968835392,
        bagging_fraction=0.7342793249627353,
        bagging_freq=2,
        min_split_gain=0.3017007245252093,
        reg_alpha=0.07886266683723922,
        reg_lambda=0.0859776680854032,
        tree_learner='serial',
        max_bin=310,
        num_threads=3,
        scale_pos_weight=3.8048327134060944
    ),
    'catboost': CatBoostClassifier(
        iterations=622,
        learning_rate=0.15620856452750326,
        depth=3,
        l2_leaf_reg=0.023092783762797234,
        random_strength=0.013271842224732251,
        bagging_temperature=6.81082830068841,
        grow_policy='SymmetricTree',
        border_count=117,
        od_wait=15,
        verbose=0,  # 학습 중 출력 비활성화
        random_state=42
    ),
    'gbc': GradientBoostingClassifier(
        n_estimators=441,
        learning_rate=0.045696033561884446,
        max_depth=7,
        min_samples_split=2,
        min_samples_leaf=2,
        subsample=0.9753636860597981,
        max_features='log2',
        loss='exponential',
        ccp_alpha=2.8658360294722587e-05,
        validation_fraction=0.27267380097000604,
        n_iter_no_change=17,
        tol=0.005721221703263734,
        min_impurity_decrease=0.04492762681125898,
        max_leaf_nodes=65
    )

}

# 소프트 보팅 모델 생성
final_voting_model = VotingClassifier(
    estimators=[('lightgbm', final_models['lightgbm']), ('catboost', final_models['catboost']),('gbc', final_models['gbc'])],
    voting='soft',
    weights=[0.20034185481390882, 0.20030143023417502, 0.2000023940758461]  # 최적 가중치 적용
)

# 학습
final_voting_model.fit(X_train, y_train)

# 예측
y_prob = final_voting_model.predict_proba(X_valid)[:, 1]
auc_score = roc_auc_score(y_valid, y_prob)

# 결과 출력
print(f"Final Model AUC Score: {auc_score:.4f}")

# 제출 파일 생성
# 테스트 데이터 예측
test_prob = final_voting_model.predict_proba(test)[:, 1]

submission['Exited'] = test_prob
submission.to_csv('origin_submission.csv',index=False)